In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [10]:
iris = load_iris()
X, y = iris.data, iris.target.reshape(-1, 1)
class_names = iris.target_names # ['setosa', 'versicolor', 'virginica']


In [11]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [14]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [8]:
model = MLP(4, 8, 3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [16]:
for epoch in range(200):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, torch.max(y_train, 1)[1]) # convert one-hot to class indices
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 1.0934
Epoch 50, Loss: 0.5966
Epoch 100, Loss: 0.5743
Epoch 150, Loss: 0.5699


In [18]:
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    _, true_labels = torch.max(y_test, 1)
    accuracy = (predicted == true_labels).float().mean()
    print("Test Accuracy:", accuracy.item())

Test Accuracy: 1.0


In [19]:
print("\nEnter flower measurements to predict Iris class:")
sepal_length = float(input("Sepal Length (cm): "))
sepal_width = float(input("Sepal Width (cm): "))
petal_length = float(input("Petal Length (cm): "))
petal_width = float(input("Petal Width (cm): "))


Enter flower measurements to predict Iris class:
Sepal Length (cm): 5
Sepal Width (cm): 6
Petal Length (cm): 4
Petal Width (cm): 3


In [20]:
user_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
user_data_scaled = scaler.transform(user_data)
user_tensor = torch.tensor(user_data_scaled, dtype=torch.float32)

In [21]:
with torch.no_grad():
 user_output = model(user_tensor)
 predicted_class = torch.argmax(user_output, dim=1).item()
print("\nPredicted Class Label:", predicted_class)
print("Predicted Class Name:", class_names[predicted_class])


Predicted Class Label: 0
Predicted Class Name: setosa
